# TRAINING

In [3]:
# 1. NVIDIA-Treiber installieren (aktuellste Version, Download von NVIDIA.com)
# 2. CUDA Toolkit installieren (https://developer.nvidia.com/cuda-downloads)
# 3. Ultralytics installieren (automatisch inkl. torch/cuda support)
!python -m pip install --upgrade pip
%pip install ultralytics --upgrade
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu128, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Nach der Installation prüfe in Python, ob CUDA verfügbar ist:

In [4]:
import torch
print(torch.cuda.is_available())  # True = CUDA bereit


True


# Pfad zur YAML Datei eintragen

Passe bei Bedarf die Parameter an (epochs, batch, imgsz …)

Starte das Skript – Training läuft!

In [5]:
from ultralytics import YOLO

# --- Einstellungen (bitte ggf. anpassen) ---

MODEL_PATH = 'yolo11n.pt'               # Nano Modell, schnell und ressourcensparend (Standard)

DATASET_YAML = r"C:\Users\Michel\Pictures\Test2_img_split\Test2_img\dataset.yaml"   # Pfad zur dataset.yaml (wie vorhin erzeugt)


### Trainings-Argumente einstellen

| Argument       | Bedeutung (einfach erklärt)                                                          | Default           |
| -------------- | ------------------------------------------------------------------------------------ | ----------------- |
| `epochs`       | Wie oft wird der ganze Datensatz "durchtrainiert"? (Mehr = besser, aber auch länger) | 100               |
| `imgsz`        | Bildgrösse, auf die vor Training skaliert wird.                                      | 640               |
| `batch`        | Wie viele Bilder werden gleichzeitig an die GPU/CPU geschickt?                       | 16                |
| `device`       | 0 = erste GPU, 1 = zweite GPU, 'cpu' = Prozessor                                     | 0                 |
| `optimizer`    | Optimierungsverfahren, das das Lernen steuert.                                       | 'auto'            |
| `lr0`          | Startwert der Lernrate                                                               | 0.01              |
| `lrf`          | Endwert der Lernrate, als Anteil von lr0                                             | 0.01              |
| `momentum`     | Schwung der Gewichtsaktualisierung (wichtig bei SGD)                                 | 0.937             |
| `weight_decay` | Bestraft zu komplexe Modelle (Überanpassungsschutz)                                  | 0.0005            |
| `patience`     | Wieviele Epochen abwarten, wenn keine Verbesserung mehr kommt?                       | 50                |
| `cos_lr`       | Nutze Cosine-Learningrate, oft für längeres, sanfteres Lernen                        | False             |
| `project`      | Ordner, in dem alles gespeichert wird                                                | 'runs/train'      |
| `name`         | Name des Experiments (wird im Projektordner gespeichert)                             | 'yolo11n\_custom' |
| `pretrained`   | Beginne mit vortrainiertem Modell (schneller & meist bessere Ergebnisse)             | True              |
| `resume`       | Falls Training abbricht, kannst du damit weitermachen                                | False             |
| `val`          | Nach jedem Durchgang das Modell testen/validieren                                    | True              |
| `workers`      | CPU-Prozesse fürs Datenladen (mehr ist schneller, je nach PC)                        | 4                 |


In [ ]:
# Trainingsparameter (alle wichtigsten Argumente, mit Defaults)
TRAIN_ARGS = {
    'epochs': 100,           # Wie oft das ganze Dataset gesehen wird (Standard: 100)
    'imgsz': 640,            # Bildgrösse, Standard ist 640 (je nach GPU bis 1280 möglich)
    'batch': 32,             # Batch-Grösse, Anzahl Bilder pro Schritt (je nach VRAM 8–32)
    'device': 0,             # GPU-Index (0=erste GPU, 'cpu'=CPU nutzen, [0,1] für mehrere GPUs)
    'optimizer': 'auto',     # Optimizer (z.B. SGD, Adam, AdamW oder 'auto' für Automatik)
    'lr0': 0.01,             # Start-Lernrate
    'lrf': 0.01,             # End-Lernrate als Faktor von lr0. Also 0.01 = 1% von lr0
    'momentum': 0.937,       # Nur für SGD, "Schwung" der Updates
    'weight_decay': 0.0005,  # Regularisierung gegen Überanpassung
    'patience': 50,          # Stoppt Training, falls Val-Loss nicht besser wird (Epochen)
    'cos_lr': False,         # Cosine-Learningrate statt linear (True/False)
    'project': 'runs/train', # Wohin die Ergebnisse gespeichert werden
    'name': 'yolo11n_custom',# Name des Experiments (wird als Ordner erzeugt)
    'pretrained': True,      # Pretrained Modell nutzen (empfohlen)
    'resume': False,         # Training fortsetzen, falls gestoppt
    'val': True,             # Nach jedem Epoch validieren (empfohlen)
    'workers': 14,            # Anzahl CPU-Worker fürs Laden (je nach PC 2–8)
    
    # Data Augmentation direkt im Training (Stärke, Methode)
    'hsv_h': 0.015,             # Farbtonverschiebung (0.0–0.5), 0=aus
    'hsv_s': 0.7,               # Sättigung (0.0–0.9)
    'hsv_v': 0.4,               # Helligkeit (0.0–0.9)
    'degrees': 0.0,             # Rotation in Grad, z. B. 10
    'translate': 0.1,           # Verschiebung als Anteil, z. B. 0.1 = 10%
    'scale': 0.5,               # Skalierung (0=aus, 0.5=±50%)
    'shear': 0.0,               # Scherung (0–2.0)
    'perspective': 0.0,         # Perspektivische Verzerrung (0–0.001)
    'flipud': 0.0,              # Vertikal spiegeln (Wahrscheinlichkeit, 0–1)
    'fliplr': 0.5,              # Horizontal spiegeln (Wahrscheinlichkeit, 0–1)
    'mosaic': 1.0,              # Mosaic-Augmentation (0–1), meist anlassen
    'mixup': 0.0,               # Mixup (0–1), Bilder kombinieren

    # Loss-Anteile für Boxen, Klassifikation, Objekt
    'box': 7.5,                 # Box-Loss-Gewichtung
    'cls': 0.5,                 # Klassifikations-Loss-Gewichtung
    'dfl': 1.5,                 # Distribution Focal Loss (0–10)

    # Early stopping deaktivieren
    'patience': 50,            # Höhere Zahl = weniger frühes Abbrechen (Standard: 50)

    # Save/checkpoints
    'save_period': 10,          # Alle x Epochen ein Modell speichern (Default=50)
    'exist_ok': True,           # Vorhandene Ergebnisse überschreiben (False=Fehler)
    'verbose': True,            # Detailierte Ausgaben (False=weniger Text)

    # Advanced: Training nur auf bestimmten Klassen (nur falls gebraucht)
    # 'classes': [0, 2],        # Nur Klasse 0 und 2 trainieren

    # Seed für Reproduzierbarkeit
    'seed': 42                 # Zufallszahlgenerator initialisiere
}

---
### Training startet mit Ausführung des nächsten Skriptes

In [12]:
# ---- Schritt 1: Modell laden ----
model = YOLO(MODEL_PATH)

# ---- Schritt 2: Training starten ----
results = model.train(
    data=DATASET_YAML,
    epochs=TRAIN_ARGS['epochs'],
    imgsz=TRAIN_ARGS['imgsz'],
    batch=TRAIN_ARGS['batch'],
    device=TRAIN_ARGS['device'],
    optimizer=TRAIN_ARGS['optimizer'],
    lr0=TRAIN_ARGS['lr0'],
    lrf=TRAIN_ARGS['lrf'],
    momentum=TRAIN_ARGS['momentum'],
    weight_decay=TRAIN_ARGS['weight_decay'],
    patience=TRAIN_ARGS['patience'],
    cos_lr=TRAIN_ARGS['cos_lr'],
    project=TRAIN_ARGS['project'],
    name=TRAIN_ARGS['name'],
    pretrained=TRAIN_ARGS['pretrained'],
    resume=TRAIN_ARGS['resume'],
    val=TRAIN_ARGS['val'],
    workers=TRAIN_ARGS['workers'],
    hsv_h=TRAIN_ARGS['hsv_h'],
    hsv_s=TRAIN_ARGS['hsv_s'],
    hsv_v=TRAIN_ARGS['hsv_v'],
    degrees=TRAIN_ARGS['degrees'],
    translate=TRAIN_ARGS['translate'],
    scale=TRAIN_ARGS['scale'],
    shear=TRAIN_ARGS['shear'],
    perspective=TRAIN_ARGS['perspective'],
    flipud=TRAIN_ARGS['flipud'],
    fliplr=TRAIN_ARGS['fliplr'],
    mosaic=TRAIN_ARGS['mosaic'],
    mixup=TRAIN_ARGS['mixup'],
    box=TRAIN_ARGS['box'],
    cls=TRAIN_ARGS['cls'],
    dfl=TRAIN_ARGS['dfl'],
    save_period=TRAIN_ARGS['save_period'],
    exist_ok=TRAIN_ARGS['exist_ok'],
    verbose=TRAIN_ARGS['verbose'],
    # classes=TRAIN_ARGS['classes'],  # Nur wenn nötig
    seed=TRAIN_ARGS['seed']
)

# Das Training wird jetzt gestartet. Je nach Hardware kann das einige Zeit dauern.
# Die Ergebnisse werden im angegebenen Projekt-Ordner gespeichert.


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     40/100      4.24G     0.7191     0.7796      1.028         91        640: 100%|██████████| 6/6 [00:01<00:00,  4.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.95it/s]

                   all         54         77      0.969      0.802        0.9      0.784



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/100      4.23G     0.6887     0.7101     0.9836         70        640: 100%|██████████| 6/6 [00:01<00:00,  4.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

...

    100/100       4.2G     0.2099      0.299     0.7867         39        640: 100%|██████████| 6/6 [00:01<00:00,  5.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.84it/s]

                   all         54         77      0.996      0.923      0.949      0.924



100 epochs completed in 0.064 hours.
Optimizer stripped from runs\train\yolo11n_custom\weights\last.pt, 5.5MB
Optimizer stripped from runs\train\yolo11n_custom\weights\best.pt, 5.5MB

Validating runs\train\yolo11n_custom\weights\best.pt...
Ultralytics 8.3.140  Python-3.10.6 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8192MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.68it/s]


                   all         54         77      0.983      0.927      0.952      0.925
                KLEBER         18         24          1      0.858      0.926      0.892
                EDDING         18         26      0.998      0.923      0.934      0.899
                TACKER         23         27      0.952          1      0.995      0.984
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\train\yolo11n_custom
